In [1]:
!pip install mat73

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import sys
sys.path.append('/content/drive/MyDrive/GTxPython')

Mounted at /content/drive


In [3]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="google.protobuf")

In [4]:
import os
import sys
os.chdir('/content/drive/MyDrive/GTxPython')
os.environ.pop("LD_LIBRARY_PATH", None)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


In [5]:
import tensorflow as tf
print(tf.test.is_built_with_cuda())
print(tf.config.list_physical_devices("GPU"))


True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
import scipy.io as sio
import mat73
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [7]:
import keras
import tensorflow as tf
from keras.saving import register_keras_serializable

In [8]:
from code_tf.preprocess import *

In [9]:
print("Python:", sys.executable)
print("TF version:", tf.__version__)
print("CUDA built with:", tf.sysconfig.get_build_info().get("cuda_version"))
print("cuDNN built with:", tf.sysconfig.get_build_info().get("cudnn_version"))

Python: /usr/bin/python3
TF version: 2.19.0
CUDA built with: 12.5.1
cuDNN built with: 9


In [10]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        print(gpus)
        tf.config.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Model

## unzip model

In [11]:
# !tar -xvzf code_tf/aws_ckpt/8000_1024_unet_2d_TBR/model.tar.gz -C code_tf/aws_ckpt/8000_1024_unet_2d_TBR/

## test model

In [12]:
@register_keras_serializable(package="custom")
def tumor_mae(y_true, y_pred):
    mask = tf.not_equal(y_true, 0.0)
    err  = tf.abs(y_true - y_pred)
    masked_err = tf.boolean_mask(err, mask)
    return tf.cond(tf.size(masked_err) > 0,
                   lambda: tf.reduce_mean(masked_err),
                   lambda: tf.constant(0.0, dtype=y_true.dtype))

In [13]:
model = keras.models.load_model("/content/drive/MyDrive/GTxPython/ckpt_unet/8000_1024_unet_2d_TBR/model.keras", safe_mode=True)

input_1 = tf.constant(np.ones((1, 101, 101, 2), dtype=np.float32))
input_2 = tf.constant(np.ones((1, 101, 101, 6, 1), dtype=np.float32))

output = model([input_1, input_2], training=False)

print(output[0].shape)
print(output[1].shape)

(1, 101, 101)
(1, 101, 101)


# Prediction

In [14]:
def draw_pred(gt_depth, pred_depth):
  fig, ax = plt.subplots(1, 3, figsize=(15, 5))
  v_min = min(np.min(gt_depth), np.min(pred_depth))
  v_max = max(np.max(gt_depth), np.max(pred_depth))
  im0 = ax[0].imshow(gt_depth, cmap='viridis', vmin=v_min, vmax=v_max)
  ax[0].set_title('Ground Truth')
  ax[1].imshow(pred_depth, cmap='viridis', vmin=v_min, vmax=v_max)
  ax[1].set_title('Prediction')
  im2 = ax[2].imshow(gt_depth - pred_depth, cmap='Purples')
  ax[2].set_title('Difference')
  cbar = fig.colorbar(im0, ax=ax[0:2], orientation='vertical')
  cbar.set_label('Depth')
  cbar_err = fig.colorbar(im2, ax=ax[2], orientation='vertical')
  cbar_err.set_label('Error')
  plt.show()

In [ ]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm

def calculate_unet_depth(
    model, data, mm=None, draw=False, p=False, device="auto", batch=8
):
    """
    Batched + GPU-ready evaluation of your UNet depth head.

    Args
    ----
    model : tf.keras.Model
        Takes [op_batch, fl_batch] and returns [pred_conc, pred_depth].
    data : dict
        data['test'] with keys: 'mu_a','mu_s','fluorescence','depth'
        Shapes typically:
          mu_a, mu_s: (N, H, W)
          fluorescence: (N, H, W, 6)
          depth: (N, H, W)
    mm : int | None
        If provided, keeps only indices i where i % 4 == (mm/2 - 1)
    draw : bool
        If True, calls draw_pred(GT, P) per example.
    p : bool
        If True, prints extra per-metric means.
    device : {"auto", "cpu", "gpu", "/CPU:0", "/GPU:0"}
        Where to run inference. "auto" -> GPU if available else CPU.
    batch : int
        Batch size for inference.
    """
    test_data = data["test"]

    # ---------- helpers ----------
    def masked_argmin_2d(A, M):
        rr, cc = np.where(M)
        if rr.size == 0:
            return None
        k = int(np.argmin(A[rr, cc]))
        return int(rr[k]), int(cc[k])

    # ---------- choose device ----------
    if device == "auto":
        device_str = "/GPU:0" if tf.config.list_physical_devices("GPU") else "/CPU:0"
    elif device.lower() == "gpu":
        device_str = "/GPU:0"
    elif device.lower() == "cpu":
        device_str = "/CPU:0"
    else:
        device_str = device  # allow explicit "/GPU:1", etc.

    # Optional: enable memory growth to avoid TF pre-allocating all VRAM
    try:
        gpus = tf.config.experimental.list_physical_devices('GPU')
        for g in gpus:
            tf.config.experimental.set_memory_growth(g, True)
    except Exception:
        pass

    # ---------- build evaluation index set ----------
    N = len(test_data["depth"])
    if mm is not None:
        d = int(mm / 2) - 1
        idx_all = [i for i in range(N) if i % 4 == d]
    else:
        idx_all = list(range(N))

    if len(idx_all) == 0:
        print("No test indices after filtering. Nothing to evaluate.")
        return {
            "total_mae": np.nan,
            "mean_min_gt_resid": np.nan,
            "mean_min_pred_resid": np.nan,
            "mean_min_diff": np.nan,
            "mean_min_dist": np.nan,
        }

    # ---------- precompute OP tensor shape ----------
    # OP: stack(mu_a, mu_s) -> (N, 2, H, W) then transpose to (N, H, W, 2)
    mu_a = test_data["mu_a"]
    mu_s = test_data["mu_s"]
    op_all = np.stack([mu_a, mu_s], axis=1).transpose(0, 2, 3, 1)  # (N, H, W, 2)

    # Fluorescence: (N, H, W, 6) -> add channel dim -> (N, H, W, 6, 1)
    fl_all = test_data["fluorescence"]
    if fl_all.ndim == 4:
        fl_all = fl_all[..., np.newaxis]  # add channel last

    # GT depth: (N, H, W)
    depth_all = test_data["depth"]

    # ---------- accumulators ----------
    count = 0
    error = 0.0
    gt_depth = []
    pred_depth = []
    diff = []
    distance = []

    # ---------- batched inference ----------
    with tf.device(device_str):
        for b_start in tqdm(range(0, len(idx_all), batch)):
            b_end = min(b_start + batch, len(idx_all))
            batch_idx = idx_all[b_start:b_end]

            # Build batch inputs
            op_batch = op_all[batch_idx]               # (B, H, W, 2)
            fl_batch = fl_all[batch_idx]               # (B, H, W, 6, 1)
            gt_batch = depth_all[batch_idx]            # (B, H, W)

            # Convert to tensors
            input_1 = tf.convert_to_tensor(op_batch, dtype=tf.float32)
            input_2 = tf.convert_to_tensor(fl_batch, dtype=tf.float32)

            # Model forward pass (no gradients)
            outputs = model([input_1, input_2], training=False)
            # outputs: [pred_conc, pred_depth]
            pred_conc_b = outputs[0].numpy()  # (B, H, W) — not used below
            pred_depth_b = outputs[1].numpy() # (B, H, W)

            # Per-sample metric computation
            for k in range(len(batch_idx)):
                GT = gt_batch[k]               # (H, W)
                P  = pred_depth_b[k]           # (H, W)
                P[GT == 0] = 0                 # ignore background in preds

                # keep your "flag" behavior
                flag = (np.mean(GT) == 10)
                if flag:
                    continue

                # masks
                M_gt   = GT > 0
                M_pred = P  > 0
                M_err  = M_gt & M_pred

                if np.any(M_gt):
                    count += np.sum(M_gt)
                    error += np.sum(np.abs(GT[M_gt] - P[M_gt]))

                # 1) Min GT depth over tumor
                ij_gt = masked_argmin_2d(GT, M_gt)
                if ij_gt is not None:
                    i_gt, j_gt = ij_gt
                    min_gt_depth = float(GT[i_gt, j_gt])
                    pred_at_gt   = float(P[i_gt, j_gt])
                    resid_at_gt  = pred_at_gt - min_gt_depth
                else:
                    min_gt_depth = np.nan
                    resid_at_gt  = np.nan

                # 3) Min predicted depth over evaluable
                ij_pred = masked_argmin_2d(P, M_err)
                if ij_pred is not None:
                    i_p, j_p = ij_pred
                    min_pred_depth = float(P[i_p, j_p])
                    gt_at_pred     = float(GT[i_p, j_p])
                    resid_at_pred  = min_pred_depth - gt_at_pred
                else:
                    min_pred_depth = np.nan
                    resid_at_pred  = np.nan

                # 4) Min |error| over evaluable pixels
                if np.any(M_err):
                    resid_field = P - GT
                    abs_resid   = np.abs(resid_field)
                    ij_err = masked_argmin_2d(abs_resid, M_err)
                    if ij_err is not None:
                        i_e, j_e = ij_err
                        min_abs_err = float(abs_resid[i_e, j_e])
                        gt_at_err   = float(GT[i_e, j_e])
                        pred_at_err = float(P[i_e, j_e])
                        _ = (min_abs_err, gt_at_err, pred_at_err)  # kept for parity
                # d and dist (handle NaNs)
                if not np.isnan(min_pred_depth) and not np.isnan(min_gt_depth):
                    d = min_pred_depth - min_gt_depth
                else:
                    d = np.nan

                if (ij_pred is not None) and (ij_gt is not None):
                    dist = np.linalg.norm(np.array(ij_pred) - np.array(ij_gt))
                else:
                    dist = np.nan

                gt_depth.append(resid_at_gt)
                pred_depth.append(resid_at_pred)
                diff.append(d)
                distance.append(dist)

                if draw:
                    try:
                        draw_pred(GT, P)
                    except NameError:
                        # draw_pred not defined in scope—silently skip
                        pass

    # ---------- summaries ----------
    total_mae = (error / count) if count > 0 else np.nan

    # Compact prints (same as before)
    if p:
        print(f"Min GT depth {np.nanmean(gt_depth)}")
        print(f"Min pred depth {np.nanmean(pred_depth)}")
        print(f"Difference {np.nanmean(diff)}")
        print(f"Distance {np.nanmean(distance)}")

    print(f"Total average tumor region depth error {total_mae}")

    return {
        "total_mae": total_mae,
        "mean_min_gt_resid": np.nanmean(gt_depth) if len(gt_depth) else np.nan,
        "mean_min_pred_resid": np.nanmean(pred_depth) if len(pred_depth) else np.nan,
        "mean_min_diff": np.nanmean(diff) if len(diff) else np.nan,
        "mean_min_dist": np.nanmean(distance) if len(distance) else np.nan,
    }

In [ ]:
def make_pred(file_path, model, scale_params, seed, normalize, mm, draw, p):
    data = load_data(file_path, scale_params, seed, normalize)
    return calculate_unet_depth(model, data, mm, draw, p)

In [34]:
scale_params = {
    'fluorescence': 10e4,
    'mu_a': 10,
    'mu_s': 1,
    'depth': 1,
    'concentration_fluor': 1,
    'reflectance': 1,
}
file_path = "/content/drive/MyDrive/GTxPython/data/finalized/ts_2d_10000_original_TBR.mat"
data = load_data("/content/drive/MyDrive/GTxPython/data/finalized/ts_2d_10000_original_TBR.mat", scale_params, 24, normalize=False)

FFFFF: [(np.float32(1.5005018e-07), np.float32(0.0022840023)), (np.float32(2.768177e-08), np.float32(0.001968136)), (np.float32(9.098925e-09), np.float32(0.0015638856)), (np.float32(3.6914662e-09), np.float32(0.0014251228)), (np.float32(3.0505463e-09), np.float32(0.0012524473)), (np.float32(2.5957803e-09), np.float32(0.001083783))]
OOOOO: [(np.float32(0.000958141), np.float32(0.032059222)), (np.float32(0.7320821), np.float32(2.083207))]


In [35]:
model = keras.models.load_model("/content/drive/MyDrive/GTxPython/ckpt_unet/original_2d_ts/model.keras", safe_mode=True)
calculate_unet_depth(model, data, mm=None, draw=False, p=True)

100%|██████████| 125/125 [00:25<00:00,  4.88it/s]

Min GT depth 0.4349835680555552
Min pred depth -0.09823728489875794
Difference 0.02791273999027908
Distance 12.196945794097848
Total average tumor region depth error 0.5286947110956465


{'total_mae': np.float64(0.5286947110956465),
 'mean_min_gt_resid': np.float64(0.4349835680555552),
 'mean_min_pred_resid': np.float64(-0.09823728489875794),
 'mean_min_diff': np.float64(0.02791273999027908),
 'mean_min_dist': np.float64(12.196945794097848)}

In [40]:
def old_calculate(model, data, mm=None, draw=False, p=False):
    test_data = data["test"]

    def masked_argmin_2d(A, M):
        rr, cc = np.where(M)
        if rr.size == 0:
            return None
        k = int(np.argmin(A[rr, cc]))
        return int(rr[k]), int(cc[k])

    count = 0
    error = 0.0
    gt_depth = []
    pred_depth = []
    diff = []
    distance = []

    for i in tqdm(range(len(test_data["depth"]))):
        if mm is not None:
            d = mm/2 - 1
            if i % 4 != d:
                continue

        # inputs
        op = np.stack([test_data['mu_a'], test_data['mu_s']], axis=1).transpose(0, 2, 3, 1)[i, ...]
        op = np.expand_dims(op, axis=0)

        fl = test_data["fluorescence"][i]
        fl = np.expand_dims(fl, axis=0)
        fl = np.expand_dims(fl, axis=-1)

        input_1 = tf.convert_to_tensor(op, dtype=tf.float32)
        input_2 = tf.convert_to_tensor(fl, dtype=tf.float32)

        output = model([input_1, input_2], training=False)


        GT = test_data["depth"][i]                          # ground truth depth
        pred_conc  = output[0][0, :, :]              # not used below but kept
        P = output[1][0, :, :].numpy()               # prediction (depth)
        P[GT == 0] = 0                                           # ignore background in preds

        flag = (np.mean(GT) == 10)

        if not flag:
            # masks
            M_gt   = GT > 0
            M_pred = P > 0
            M_err  = M_gt & M_pred

            mae_tumor = np.mean(np.abs(GT[M_gt] - P[M_gt])) if np.any(M_gt) else np.nan
            # print(f"average tumor region depth error {mae_tumor}") if p else None

            count += np.sum(M_gt)
            error += np.sum(np.abs(GT[M_gt] - P[M_gt]))

            # 1) Min GT depth (over tumor region)
            ij_gt = masked_argmin_2d(GT, M_gt)
            if ij_gt is not None:
                i_gt, j_gt = ij_gt
                min_gt_depth   = float(GT[i_gt, j_gt])
                pred_at_gt     = float(P[i_gt, j_gt])
                resid_at_gt    = pred_at_gt - min_gt_depth

            # 3) Min predicted depth, but only where error is defined (M_err)
            ij_pred = masked_argmin_2d(P, M_err)
            if ij_pred is not None:
                i_p, j_p = ij_pred
                min_pred_depth = float(P[i_p, j_p])
                gt_at_pred     = float(GT[i_p, j_p])
                resid_at_pred  = min_pred_depth - gt_at_pred

            # 4) Min |error| over evaluable pixels
            if np.any(M_err):
                resid_field  = P - GT
                abs_resid    = np.abs(resid_field)
                ij_err = masked_argmin_2d(abs_resid, M_err)
                i_e, j_e = ij_err
                min_abs_err = float(abs_resid[i_e, j_e])
                gt_at_err   = float(GT[i_e, j_e])
                pred_at_err = float(P[i_e, j_e])
                resid_at_err= pred_at_err - gt_at_err

            d = min_pred_depth - min_gt_depth
            dist = np.linalg.norm(np.array(ij_pred) - np.array(ij_gt))

            gt_depth.append(resid_at_gt)
            pred_depth.append(resid_at_pred)
            diff.append(d)
            distance.append(dist)

            draw_pred(GT, P) if draw else None

    if p:
        print(f"Min GT depth {np.mean(gt_depth)}")
        print(f"Min pred depth {np.mean(pred_depth)}")
        print(f"Difference {np.mean(diff)}")
        print(f"Distance {np.mean(distance)}")


    # global MAE over all tumor pixels seen
    total_mae = (error / count) if count > 0 else np.nan
    print(f"Total average tumor region depth error {total_mae}")


In [41]:
old_calculate(model, data, mm=None, draw=False, p=True)

100%|██████████| 1000/1000 [02:09<00:00,  7.73it/s]

Min GT depth 0.43498385573737325
Min pred depth -0.09823702824115753
Difference 0.02791299664787948
Distance 12.196945794097848
Total average tumor region depth error 0.5286948434311817
